Pierwszym krokiem jest stworzenie populacji losowych ciągów bitowych. Możemy użyc wartości logicznych "True" i "False", wartości string '0' i '1' lub wartości int 0 i 1. W tym przykładzie, wykorzystamy wartości liczbowe.

Możemy wygenerować tablicę wartości całkowitych w zadanym zakresie za pomocą funkcji randint() a następnie określić zakres jako wartości zaczynąjące się od 0 a kończące na 2. Naszymi wynikami bedą zera i jedynki. Aby przedstawić przykład w jak najprostszy sposób, proponowane rozwiązanie zaprezentuję jako listę zamiast tablicy NumPy.

Poniżej prezentuję przykład utworzenia początkowej populacji losowej ciągu bitów. 'n_pop' jest hiperparametrem kontrolującym wielkość populacji, a 'n_bits' jest hiperparametrem definiującym liczbę bitów w pojedynczym rozwiązaniu.

In [30]:
# genetic algorithm search of the one max optimization problem
from numpy.random import randint
from numpy.random import rand

In [31]:
n_bits= 5
n_pop = 2
# początkowa populacja losowego ciągu bitowego
pop = [np.random.randint(0, 2, n_bits).tolist() for _ in range(n_pop)]
print(pop)

[[1, 0, 1, 0, 0], [0, 0, 0, 0, 0]]


Następnie możemy wyliczyć stałą liczbę iteracji algorytmu, w tym przypadku za tą stałą będzie odpowiedzialny hiperparametr o nazwie 'n_iter'

In [21]:
# enumerate generations
 for gen in range(n_iter):

IndentationError: unexpected indent (1433582520.py, line 2)

Pierwszym krokiem w jednej iteracji algorytmu jest ocena wszystkich potencjalnych rozwiązań.

Użyjemy funkcji o nazwie cel() jako ogólnej funkcji celu i wywołamy ją, aby uzyskać wynik sprawności, który zminimalizujemy.


In [22]:
#ocena wszystkich kandydatów w populacji
wyniki = [cel(c) for c in pop]

Następnie możemy wybrać rodziców, którzy zostaną wykorzystani do stworzenia dzieci.

Procedurę selekcji można zaimplementować jako funkcję, która pobiera populację i zwraca jednego wybranego rodzica. Wartość k jest ustalona na 3 z domyślnym argumentem.

In [23]:
# selekcja
def selekcja(pop, wyniki, k=3):
	# pierwszy losowy wybór
	selekcja_ix = np.random.randint(len(pop))
	for ix in np.random.randint(0, len(pop), k-1):
		# sprawdzanie czy wybrany został lepszy 
		if wyniki[ix] < wyniki[selekcja_ix]:
			selekcja_ix = ix
	return pop[selekcja_ix]

Możemy następnie wywołać tę funkcję jeden raz dla każdej pozycji w populacji, aby utworzyć listę rodziców.

In [24]:
# wybór rodziców
wybrani = [selekcja(pop, wyniki) for _ in range(n_pop)]

Teraz możemy stworzyć kolejną generację

Stworzenie kolejnej generacji najpierw wymaga funkcji do wykonania krzyżowania. Ta funkcja będzie uwzględniać dwoje rodziców i współczynnik krzyżowania. Współczynnik krzyżowania to hiperparametr określający, czy krzyżowanie jest wykonywane, czy nie, a jeśli nie, elementy nadrzędne są kopiowane do następnej generacji. Ten parametr to współczynnik prawdopodobieństwa i zazwyczaj ma dużą wartość bliską 1,0.

Poniższa funkcja krzyzowanie() implementuje krzyżowanie poprzez losowanie liczby z zakresu [0,1] w celu ustalenia, czy krzyżowanie jest wykonywane, a następnie wybiera prawidłowy punkt podziału, jeśli ma zostać przeprowadzone krzyżowanie.

In [25]:
# krzyżowanie dwóch rodziców w celu wygenerowania dwojga potomków
def krzyzowanie(p1, p2, r_cross):
    # potomkowie domyślnie są kopią rodziców
	c1, c2 = p1.copy(), p2.copy()
	# sprawdzanie w celu rekombinacji
	if rand() < r_cross:
		# select crossover point that is not on the end of the string
        # wybór punktu krzyżowania, który nie znajduje się na końcu ciągu
		pt = np.random.randint(1, len(p1)-2)
		# wykonanie krzyżowania
		c1 = p1[:pt] + p2[pt:]
		c2 = p2[:pt] + p1[pt:]
	return [c1, c2]

Potrzebujemy również funkcji do przeprowadzenia mutacji.

Ta procedura po prostu odwraca bity z niskim prawdopodobieństwem kontrolowanym przez hiperparametr „r_mut”.

In [26]:
# operator mutacji
def mutacja(bitstring, r_mut):
	for i in range(len(bitstring)):
		# sprawdzanie w celu przeprowadzenie mutacji
		if rand() < r_mut:
			# odwrócenie bitu
			bitstring[i] = 1 - bitstring[i]

Aby utworzyć listę dzieci, które zostaną wykorzystane jako następne pokolenie, możemy zapętlić program przez listę rodziców, wywołując w razie potrzeby funkcje krzyżowania i mutacji

In [27]:
...
# stworzenie następnej generacji
dzieci = list()
for i in range(0, n_pop, 2):
	# połączenie wybranych rodziców w pary
	p1, p2 = wybrani[i], wybrani[i+1]
	# krzyżowanie i mutacja
	for c in krzyzowanie(p1, p2, r_cross):
		# utacja
		mutacja(c, r_mut)
		# store for next generation
		dzieci.append(c)

NameError: name 'r_cross' is not defined

Możemy połączyć to wszystko w funkcję o nazwie Genetic_algorithm(), która pobiera nazwę funkcji celu oraz hiperparametry wyszukiwania i zwraca najlepsze rozwiązanie znalezione podczas wyszukiwania.

In [28]:
# genetic algorithm
def algorytm_genetyczny(cel, n_bits, n_iter, n_pop, r_krzyz, r_mut):
	# initial population of random bitstring
	pop = [np.random.randint(0, 2, n_bits).tolist() for _ in range(n_pop)]
	# keep track of best solution
	best, best_eval = 0, cel(pop[0])
	# enumerate generations
	for gen in range(n_iter):
		# evaluate all candidates in the population
		wyniki = [cel(c) for c in pop]
		# check for new best solution
		for i in range(n_pop):
			if wyniki[i] < best_eval:
				best, best_eval = pop[i], wyniki[i]
				print(">%d, new best f(%s) = %.3f" % (gen,  pop[i], wyniki[i]))
		# select parents
		wybrani = [selekcja(pop, wyniki) for _ in range(n_pop)]
		# create the next generation
		dzieci = list()
		for i in range(0, n_pop, 2):
			# get selected parents in pairs
			p1, p2 = wybrani[i], wybrani[i+1]
			# crossover and mutation
			for c in krzyzowanie(p1, p2, r_krzyz):
				# mutation
				mutacja(c, r_mut)
				# store for next generation
				dzieci.append(c)
		# replace population
		pop = dzieci
	return [best, best_eval]



In [20]:
print(best)

NameError: name 'best' is not defined

In [33]:

# Funkcja celu - najprostsza funkcja celu
def cel(x):
    # Prosta funkcja celu - suma wszystkich bitów w ciągu
    return -sum(x)

# Funkcja selekcji - turniejowa selekcja
def selekcja(pop, wyniki, k=3):
    # Pierwszy losowy wybór
    selekcja_ix = np.random.randint(len(pop))
    for ix in [np.random.randint(0, len(pop)-1) for _ in range(k-1)]:
        # Sprawdzanie, czy wybrany został lepszy
        if wyniki[ix] < wyniki[selekcja_ix]:
            selekcja_ix = ix
    return pop[selekcja_ix]

# Funkcja krzyżowania - jednopunktowe krzyżowanie
def krzyzowanie(p1, p2, r_cross):
    # Potomkowie domyślnie są kopią rodziców
    c1, c2 = p1.copy(), p2.copy()
    # Sprawdzanie w celu rekombinacji
    if rand() < r_cross:
        # Wybór punktu krzyżowania, który nie znajduje się na końcu ciągu
        pt = np.random.randint(1, len(p1)-2)
        # Wykonanie krzyżowania
        c1 = p1[:pt] + p2[pt:]
        c2 = p2[:pt] + p1[pt:]
    return [c1, c2]

# Operator mutacji
def mutacja(bitstring, r_mut):
    for i in range(len(bitstring)):
        # Sprawdzanie w celu przeprowadzenia mutacji
        if rand() < r_mut:
            # Odwrócenie bitu
            bitstring[i] = 1 - bitstring[i]

# Algorytm genetyczny
def algorytm_genetyczny(cel, n_bits, n_iter, n_pop, r_krzyz, r_mut):
    # Początkowa populacja losowych ciągów bitów
    pop = [np.random.randint(0, 2, n_bits).tolist() for _ in range(n_pop)]
    # Śledzenie najlepszego rozwiązania
    best, best_eval = 0, cel(pop[0])
    # Przechodzenie przez generacje
    for gen in range(n_iter):
        # Ocena wszystkich kandydatów w populacji
        wyniki = [cel(c) for c in pop]
        # Sprawdzanie nowego najlepszego rozwiązania
        for i in range(n_pop):
            if wyniki[i] < best_eval:
                best, best_eval = pop[i], wyniki[i]
                print(">%d, new best f(%s) = %.3f" % (gen,  pop[i], wyniki[i]))
        # Selekcja rodziców
        wybrani = [selekcja(pop, wyniki) for _ in range(n_pop)]
        # Tworzenie kolejnej generacji
        dzieci = list()
        for i in range(0, n_pop, 2):
            # Pobieranie wybranych rodziców parami
            p1, p2 = wybrani[i], wybrani[i+1]
            # Krzyżowanie i mutacja
            for c in krzyzowanie(p1, p2, r_krzyz):
                # Mutacja
                mutacja(c, r_mut)
                # Przechowywanie dla następnej generacji
                dzieci.append(c)
        # Zastępowanie populacji
        pop = dzieci
    return [best, best_eval]

# Przykładowe wywołanie funkcji algorytmu genetycznego
n_bits = 10
n_iter = 50
n_pop = 20
r_krzyz = 0.7
r_mut = 1.0 / float(n_bits)

wynik = algorytm_genetyczny(cel, n_bits, n_iter, n_pop, r_krzyz, r_mut)
print("Najlepsze rozwiązanie:", wynik[0])
print("Wartość funkcji celu dla najlepszego rozwiązania:", wynik[1])


>1, new best f([1, 0, 1, 1, 1, 0, 1, 0, 1, 1]) = -7.000
>1, new best f([1, 0, 1, 1, 1, 1, 0, 1, 1, 1]) = -8.000
>2, new best f([1, 1, 1, 1, 1, 1, 1, 0, 1, 1]) = -9.000
>4, new best f([1, 1, 1, 1, 1, 1, 1, 1, 1, 1]) = -10.000
Najlepsze rozwiązanie: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Wartość funkcji celu dla najlepszego rozwiązania: -10


OneMax

In [35]:
# objective function
def onemax(x):
	return -sum(x)

In [36]:
...
# define the total iterations
n_iter = 100
# bits
n_bits = 20
# define the population size
n_pop = 100
# crossover rate
r_cross = 0.9
# mutation rate
r_mut = 1.0 / float(n_bits)

In [48]:
...
# perform the genetic algorithm search
best, score = algorytm_genetyczny(onemax, n_bits, n_iter, n_pop, r_cross, r_mut)
print('Done!')
print('f(%s) = %f' % (best, score))

>0, new best f([0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1]) = -12.000
>0, new best f([1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1]) = -13.000
>0, new best f([1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1]) = -14.000
>1, new best f([1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1]) = -15.000
>1, new best f([1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1]) = -17.000
>3, new best f([1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1]) = -18.000
>6, new best f([1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]) = -19.000
>8, new best f([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]) = -20.000
Done!
f([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]) = -20.000000


Continous Function Optimization

In [49]:
# objective function
def objective(x):
	return x[0]**2.0 + x[1]**2.0

In [50]:
...
# define range for input
bounds = [[-5.0, 5.0], [-5.0, 5.0]]

In [58]:
...
# define the total iterations
n_iter = 100
# bits per variable
n_bits = 16
# define the population size
n_pop = 100
# crossover rate
r_cross = 0.9

In [52]:
...
# mutation rate
r_mut = 1.0 / (float(n_bits) * len(bounds))

In [53]:
...
# initial population of random bitstring
pop = [randint(0, 2, n_bits*len(bounds)).tolist() for _ in range(n_pop)]

In [54]:
# decode bitstring to numbers
def decode(bounds, n_bits, bitstring):
	decoded = list()
	largest = 2**n_bits
	for i in range(len(bounds)):
		# extract the substring
		start, end = i * n_bits, (i * n_bits)+n_bits
		substring = bitstring[start:end]
		# convert bitstring to a string of chars
		chars = ''.join([str(s) for s in substring])
		# convert string to integer
		integer = int(chars, 2)
		# scale integer to desired range
		value = bounds[i][0] + (integer/largest) * (bounds[i][1] - bounds[i][0])
		# store
		decoded.append(value)
	return decoded

In [55]:
...
# decode population
decoded = [decode(bounds, n_bits, p) for p in pop]
# evaluate all candidates in the population
scores = [objective(d) for d in decoded]

In [ ]:
# perform the genetic algorithm search
best, score = genetic_algorithm(objective, bounds, n_bits, n_iter, n_pop, r_cross, r_mut)
print('Done!')
decoded = decode(bounds, n_bits, best)
print('f(%s) = %f' % (decoded, score))